<a href="https://colab.research.google.com/github/hafsaJChaudhry/Object-Detection-from-Video/blob/main/soccer_game.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First, make sure you are running on GPU and not cpu. cpu will take too long and will make my poor laptop overheat :( sad.... go to runtime downbar and click "change runtime type"

In [1]:
#check if you have access to GPU
!nvidia-smi

Tue Jan  9 05:04:36 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

download YOLOv8

you can also download a specific version like

    !pip install ultralytics 8.1.0

In [ ]:
%pip install ultralytics supervision lapx

below method is a faster way of getting the bounding box video with just 1 line of code. it outputs the frame by frame detection w/ type and also 1800 frames but in 10 seconds instead of 10 mins

In [4]:
#using a pre-trained model: yolov8m.pt ..... now i used 8x.pt to stay consistent with zarrar
# there are diff versions, but 8m is the middle one (nano, max)
#using middle one bc the diff sizes have a trade off for speed
#then we are gonna pass in the source by copying the path of our mp4
!yolo detect predict model=yolov8x.pt source="/content/sample_soccer_2.mp4"

100% 131M/131M [00:00<00:00, 234MB/s]
Ultralytics YOLOv8.0.238 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs

WARNING ⚠️ NMS time limit 0.550s exceeded
video 1/1 (1/1800) /content/sample_soccer_2.mp4: 384x640 18 persons, 125.8ms
video 1/1 (2/1800) /content/sample_soccer_2.mp4: 384x640 18 persons, 48.6ms
video 1/1 (3/1800) /content/sample_soccer_2.mp4: 384x640 18 persons, 48.5ms
video 1/1 (4/1800) /content/sample_soccer_2.mp4: 384x640 18 persons, 48.5ms
video 1/1 (5/1800) /content/sample_soccer_2.mp4: 384x640 17 persons, 38.1ms
video 1/1 (6/1800) /content/sample_soccer_2.mp4: 384x640 18 persons, 33.8ms
video 1/1 (7/1800) /content/sample_soccer_2.mp4: 384x640 17 persons, 32.9ms
video 1/1 (8/1800) /content/sample_soccer_2.mp4: 384x640 16 persons, 33.1ms
video 1/1 (9/1800) /content/sample_soccer_2.mp4: 384x640 17 persons, 30.9ms
video 1/1 (10/1800) /content/sample_soccer_2.mp4: 384x640 17 p

once above is executed, at the end it outputs where the results are saved. "Results saved to runs/detect/predict"

> but it is too big to download. so we need to compress it to view the file

**syntax:** !ffmpeg -i {"path_of_what_to_compress"} -vcode libx264 {"name_of_new_compresses_mp4"}

In [ ]:
!ffmpeg -i {"/content/runs/detect/predict/sample_soccer_2.avi"} -vcodec libx264 {"final.mp4"}

display bounding box video in google colab

In [ ]:
!wget http://commondatastorage.googleapis.com/gtv-videos-bucket/sample/VolkswagenGTIReview.mp4
!mv VolkswagenGTIReview.mp4 video.mp4

!pip install -U kora

In [8]:
from IPython.display import HTML
from kora.drive import upload_public

url = upload_public('final.mp4')

#show video
HTML(f"""<video src={url} width=500 controls/>""")

each player has a boundig box and is detected - accomplished!

>the numbers represent probability of what the model detected as type. so most of our players were detected as "person". some of the objects were detected as "sports ball". the probability changes as visual increases/decreases. Also looks like it decreases as it goes into another bounding box

additional info to know:
>yolo identifies person object as 0 and sports item object as 32
to get list of all yolo object identification numbers run this code:
    
    file_name = "../usr/local/lib/python3.8/dist-packages/ultralytics/yolo/data/datasets/coco8.yaml"
    with open(file_name , "r") as stream:
     names = yaml.safe_load(stream)["names"]
    names

Lets also detect with ellipses - sports mode

In [9]:
import ultralytics
import supervision as sv
from supervision.utils.notebook import plot_image as plot_sv_image
import cv2
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import numpy as np

sv.get_video_frames_generator
sv.VideoInfo
sv.VideoSink
sv.ColorPalette
sv.Detections
sv.BoxAnnotator

from IPython import display
display.clear_output()

In [10]:
# settings
MODEL = "yolov8x.pt"

In [11]:
from ultralytics import YOLO

In [12]:
model = YOLO(MODEL)
model.fuse()

YOLOv8x summary (fused): 268 layers, 68200608 parameters, 0 gradients, 257.8 GFLOPs


In [13]:
# dict maping class_id to class_name
CLASS_NAMES_DICT = model.model.names
# class_ids of interest - person object, sports item object
CLASS_ID = [0, 32]

In [17]:
SOURCE_VIDEO_PATH = "/content/sample_soccer_2.mp4"

# class_ids of interest - person object, sports item object
CLASS_ID = [0, 32]

# create VideoInfo instance
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)

# open target video file
TARGET_VIDEO_PATH2 = 'ellipse.mp4'

In [18]:
#ellipses
ell_annotator = sv.EllipseAnnotator()
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)
with sv.VideoSink(TARGET_VIDEO_PATH2, video_info) as sink:
    # loop over video frames
    for frame in tqdm(generator, total=video_info.total_frames):
        # model prediction on single frame and conversion to supervision Detections
        results = model.track(frame, persist=True, verbose=False)
        detections = sv.Detections.from_ultralytics(results[0])
        detections = detections[np.isin(detections.class_id, CLASS_ID)]
        annotated_frame = ell_annotator.annotate(
            scene=frame,
            detections=detections
        )
        sink.write_frame(annotated_frame)

  0%|          | 0/1800 [00:00<?, ?it/s]

compress ellipse video so we can view

In [ ]:
!ffmpeg -i {"/content/ellipse.mp4"} -vcodec libx264 {"ellipse_f.mp4"}

display elipse video in google colab

In [22]:
url2 = upload_public('ellipse_f.mp4')

#show video
HTML(f"""<video src={url2} width=500 controls/>""")

Issues we can identify - the soccer ball is not getting picked up in every frame

solution - we need to train our own model

now we need to TRACK each player

aka looking at multiple frames of a video and recognizing that the object in 1 frame is the same as an object in the subsequent frames

lets track with unique id assigned to every bounding box